In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast
import sys

# Load Datasets

In [2]:
dataset_a1_2_path = "/home/pbiel/repos/jask/analytics/reassignments/a1_2.csv"
dataset_a1_2 = pd.read_csv(dataset_a1_2_path)

# Utility Functions

In [3]:
def get_basic_stats(df):
    print(f"\n=== BASIC STATISTICS ===")
    print(f"Total reassignments: {len(df):,}")
    print(f"Unique processes moved: {df['ProcessID'].nunique():,}")
    print(f"Unique services: {df['Service'].nunique():,}")
    print(f"Time span: {(df['Timestamp'].max() - df['Timestamp'].min()) / 1000:.2f} seconds")

    # Calculate process sizes (sum of requirements)
    def parse_requirements(req_str):
        try:
            return ast.literal_eval(req_str)
        except:
            return [0, 0, 0, 0]
        
    df['Requirements_List'] = df['Requirements'].apply(parse_requirements)
    df['ProcessSize'] = df['Requirements_List'].apply(sum)
    
    print(f"\n=== PROCESS SIZE ANALYSIS ===")
    print(f"Average process size: {df['ProcessSize'].mean():.0f}")
    print(f"Median process size: {df['ProcessSize'].median():.0f}")
    print(f"Min process size: {df['ProcessSize'].min():.0f}")
    print(f"Max process size: {df['ProcessSize'].max():.0f}")

    # Analyze moves per process
    moves_per_process = df['ProcessID'].value_counts()
    print(f"\n=== MOVEMENT FREQUENCY ===")
    print(f"Processes moved once: {(moves_per_process == 1).sum():,}")
    print(f"Processes moved multiple times: {(moves_per_process > 1).sum():,}")
    print(f"Most moved process: Process {moves_per_process.index[0]} with {moves_per_process.iloc[0]} moves")

    # Analyze by process size quartiles
    size_quartiles = df['ProcessSize'].quantile([0.25, 0.5, 0.75])
    print(f"\n=== MOVEMENT BY PROCESS SIZE ===")
    print(f"25th percentile size: {size_quartiles[0.25]:.0f}")
    print(f"50th percentile size: {size_quartiles[0.5]:.0f}")
    print(f"75th percentile size: {size_quartiles[0.75]:.0f}")

    # Categorize processes by size
    df['SizeCategory'] = pd.cut(
        df['ProcessSize'], 
        bins = [
            0, 
            size_quartiles[0.25], 
            size_quartiles[0.5],
            size_quartiles[0.75], 
            float('inf')
        ],
        labels = [
            'Small', 
            'Medium', 
            'Large', 
            'XLarge'
        ]
    )
    
    size_movement_count = df.groupby('SizeCategory').size()
    print("\nMovements by size category:")
    for category, count in size_movement_count.items():
        print(f"  {category}: {count:,} moves ({count/len(df)*100:.1f}%)")
    
    # Analyze improvement over time
    print(f"\n=== IMPROVEMENT ANALYSIS ===")
    print(f"Initial improvement: {df['Improvement'].iloc[0]:.4f}%")
    print(f"Final improvement: {df['Improvement'].iloc[-1]:.4f}%")
    print(f"Total improvement gain: {df['Improvement'].iloc[-1] - df['Improvement'].iloc[0]:.4f}%")
    
    # Analyze move cost distribution
    print(f"\n=== MOVE COST ANALYSIS ===")
    print(f"Average move cost: {df['MoveCost'].mean():.2f}")
    print(f"Median move cost: {df['MoveCost'].median():.2f}")
    print(f"Move cost distribution:")
    move_cost_counts = df['MoveCost'].value_counts().sort_index()
    for cost, count in move_cost_counts.head(10).items():
        print(f"  Cost {cost}: {count:,} moves ({count/len(df)*100:.1f}%)")
    
    # Analyze temporal patterns
    print(f"\n=== TEMPORAL PATTERNS ===")
    df['TimeFromStart'] = (df['Timestamp'] - df['Timestamp'].min()) / 1000  # Convert to seconds
    
    # Divide into time periods
    time_periods = pd.cut(df['TimeFromStart'], bins=5, labels=['Period1', 'Period2', 'Period3', 'Period4', 'Period5'])
    period_stats = df.groupby(time_periods).agg({
        'ProcessSize': 'mean',
        'MoveCost': 'mean',
        'ProcessID': 'count'
    })
    
    print("\nAverage process size moved per time period:")
    for period, stats in period_stats.iterrows():
        print(f"  {period}: {stats['ProcessSize']:.0f} (avg size), {stats['ProcessID']:,} moves")

# Dataset A

In [4]:
get_basic_stats(dataset_a1_2)


=== BASIC STATISTICS ===
Total reassignments: 169,711
Unique processes moved: 881
Unique services: 862
Time span: 8.62 seconds

=== PROCESS SIZE ANALYSIS ===
Average process size: 121629
Median process size: 57721
Min process size: 3244
Max process size: 4825692

=== MOVEMENT FREQUENCY ===
Processes moved once: 5
Processes moved multiple times: 876
Most moved process: Process 423 with 505 moves

=== MOVEMENT BY PROCESS SIZE ===
25th percentile size: 24271
50th percentile size: 57721
75th percentile size: 153593

Movements by size category:
  Small: 42,571 moves (25.1%)
  Medium: 42,638 moves (25.1%)
  Large: 42,118 moves (24.8%)
  XLarge: 42,384 moves (25.0%)

=== IMPROVEMENT ANALYSIS ===
Initial improvement: 0.0000%
Final improvement: 26.7601%
Total improvement gain: 26.7601%

=== MOVE COST ANALYSIS ===
Average move cost: 1.00
Median move cost: 1.00
Move cost distribution:
  Cost 1: 169,711 moves (100.0%)

=== TEMPORAL PATTERNS ===

Average process size moved per time period:
  Perio

/tmp/ipykernel_216837/266750007.py:56: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  size_movement_count = df.groupby('SizeCategory').size()
/tmp/ipykernel_216837/266750007.py:82: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  period_stats = df.groupby(time_periods).agg({
